In [1]:
!pip install --upgrade gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827697 sha256=11697e7d01c6bd0b29afdf3115faafaee7c69d8324c2edd66fc343a0b090dd00
  Stored in directory: /root/.cache/pip/wheels/1c/77/9e/9af5470201a0b0543937933ee99ba884cd237d2faefe8f4d37
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [11]:
import random

class Maze:
    def __init__(self, n):
        """
        Initialize the maze of size (n+2) x (n+2). This includes a border of walls
        around an n x n interior. If n is even, also carves row n and column n
        so they're not fully blocked.
        """
        self.n = n
        self.size = n + 2
        self.maze = [['#' for _ in range(self.size)] for _ in range(self.size)]

        # Two sets for storing coordinates of walls and blanks
        self.wall_coords = set()
        self.blank_coords = set()

        # List to store the station coordinates
        self.stations = []

        # Lists to store taxi and passenger information
        self.taxi_location = []
        self.pass_idx = None  # Station index (0-3) for passenger location
        self.dest_idx = None  # Station index (0-3) for destination
        self.passenger_loc = None  # Actual coordinates of passenger location
        self.destination = None  # Actual coordinates of destination
        self.passenger_picked_up = False  # Track if passenger is in taxi
        self.fuel = 5000  # Initialize fuel

        # Generate the maze on initialization
        self._generate_maze()

        # Fill in the wall_coords and blank_coords sets based on the final maze
        self._populate_coordinate_sets()

        # Select the stations
        self._select_stations()

        # Select taxi location and passenger information
        self._select_taxi_and_passenger()

    def _generate_maze(self):
        """
        Internal method to carve out the maze using a DFS approach.
        """
        # 1) Mark interior carveable cells
        for row in range(1, self.n + 1):
            for col in range(1, self.n + 1):
                # Standard "odd cell" carve
                if row % 2 == 1 and col % 2 == 1:
                    self.maze[row][col] = ' '
                # If n is even, also open up the entire row n and col n
                elif self.n % 2 == 0:
                    if row == self.n or col == self.n:
                        self.maze[row][col] = ' '

        # 2) Perform DFS from the top-left carveable cell (1, 1)
        visited = set()

        def dfs(r, c):
            visited.add((r, c))
            for (nr, nc) in self._neighbors(r, c):
                if (nr, nc) not in visited:
                    # Carve a path between (r, c) and (nr, nc)
                    wall_r = (r + nr) // 2
                    wall_c = (c + nc) // 2
                    self.maze[wall_r][wall_c] = ' '
                    dfs(nr, nc)

        # Run DFS
        dfs(1, 1)

    def _neighbors(self, r, c):
        """
        Yield neighbor cells that are 2 steps away in the four
        cardinal directions, randomly shuffled.
        """
        directions = [(-2, 0), (2, 0), (0, -2), (0, 2)]
        random.shuffle(directions)
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            # Ensure the neighbor is valid and currently carved (i.e., ' ').
            if 1 <= nr <= self.n and 1 <= nc <= self.n and self.maze[nr][nc] == ' ':
                yield nr, nc

    def _populate_coordinate_sets(self):
        """
        Populate wall_coords and blank_coords based on the final
        layout in self.maze.
        """
        for r in range(self.size):
            for c in range(self.size):
                if self.maze[r][c] == '#':
                    self.wall_coords.add((r, c))
                else:
                    self.blank_coords.add((r, c))

    def _select_stations(self):
        """
        Randomly selects 4 blank cells to serve as stations.
        Stores their coordinates in self.stations as a list of [row, col] lists.
        """
        # Convert blank_coords to a list for random sampling
        blank_list = list(self.blank_coords)

        # Randomly select 4 coordinates (or fewer if there aren't enough blanks)
        num_stations = min(4, len(blank_list))
        selected_coords = random.sample(blank_list, num_stations)

        # Convert each tuple to a list and store in self.stations
        self.stations = [list(coord) for coord in selected_coords]

    def _select_taxi_and_passenger(self):
        """
        Randomly selects a blank cell for the taxi (not a station).
        Randomly selects two different stations for passenger location and destination.
        """
        # Get blank cells that are not stations
        station_coords = {tuple(station) for station in self.stations}
        available_coords = list(self.blank_coords - station_coords)

        # Select taxi location from available blank cells
        if available_coords:
            taxi_coord = random.choice(available_coords)
            self.taxi_location = list(taxi_coord)
        else:
            self.taxi_location = []

        # Select passenger location and destination from different stations
        if len(self.stations) >= 2:
            # Randomly select two different station indices
            station_indices = list(range(len(self.stations)))
            selected_indices = random.sample(station_indices, 2)

            self.pass_idx = selected_indices[0]
            self.dest_idx = selected_indices[1]

            # Store the actual coordinates
            self.passenger_loc = self.stations[self.pass_idx]
            self.destination = self.stations[self.dest_idx]
        else:
            # Fallback if there are fewer than 2 stations
            self.pass_idx = None
            self.dest_idx = None
            self.passenger_loc = None
            self.destination = None

    def step(self, action):
        """
        Execute an action and update the environment state.

        Args:
            action (int):
                0: Move south
                1: Move north
                2: Move east
                3: Move west
                4: Pick up passenger
                5: Drop off passenger

        Returns:
            tuple: (observation, reward, done, False, False)
                - observation: Current state
                - reward: Reward for the action
                - done: Whether the episode is terminated
                - False: Placeholder
                - False: Placeholder
        """
        # Initialize reward (each step costs -0.1)
        reward = -0.1

        # Reduce fuel by 1
        self.fuel -= 1

        # Check if fuel is empty
        if self.fuel <= 0:
            return self.get_state(), reward - 10, True, False, False  # Apply fuel exhaustion penalty

        # Movement actions
        if 0 <= action <= 3:
            # Movement directions as (dr, dc) for [south, north, east, west]
            directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
            dr, dc = directions[action]

            # Calculate new position
            new_r, new_c = self.taxi_location[0] + dr, self.taxi_location[1] + dc

            # Check if the new position is a wall
            if (new_r, new_c) in self.wall_coords:
                reward -= 5  # Penalty for hitting a wall
                return self.get_state(), reward, False, False, False

            # Update taxi location
            self.taxi_location = [new_r, new_c]

        # Pick up passenger
        elif action == 4:
            # Check if taxi is at passenger location and passenger isn't already picked up
            if not self.passenger_picked_up and self.taxi_location == self.passenger_loc:
                self.passenger_picked_up = True
            else:
                reward -= 10  # Penalty for incorrect pickup

        # Drop off passenger
        elif action == 5:
            # Check if taxi is at destination and passenger is in the taxi
            if self.passenger_picked_up and self.taxi_location == self.destination:
                self.passenger_picked_up = False
                reward += 50  # Reward for successful delivery

                # Randomize new passenger and destination
                self._randomize_passenger_and_destination()
            else:
                reward -= 10  # Penalty for incorrect dropoff

        # Return the observation, reward, done flag, and two placeholders
        return self.get_state(), reward, False, False, False

    def _randomize_passenger_and_destination(self):
        """
        Randomly selects new passenger and destination stations.
        The passenger and destination will be at different stations.
        """
        if len(self.stations) >= 2:
            # Randomly select two different station indices
            station_indices = list(range(len(self.stations)))
            selected_indices = random.sample(station_indices, 2)

            self.pass_idx = selected_indices[0]
            self.dest_idx = selected_indices[1]

            # Store the actual coordinates
            self.passenger_loc = self.stations[self.pass_idx]
            self.destination = self.stations[self.dest_idx]

            # Reset passenger status
            self.passenger_picked_up = False

    def __str__(self):
        """
        Allows printing the maze object directly with `print(maze)`.
        """
        return "\n".join("".join(row) for row in self.maze)

    def get_state(self):
        """
        Returns the current state representation as a tuple containing:
        - Taxi position
        - Station coordinates
        - Information about obstacles in cardinal directions
        - Information about passenger and destination visibility

        Returns:
            tuple: State representation
        """
        taxi_row, taxi_col = self.taxi_location

        # Check for obstacles in each direction
        obstacle_north = int((taxi_row - 1, taxi_col) in self.wall_coords)
        obstacle_south = int((taxi_row + 1, taxi_col) in self.wall_coords)
        obstacle_east = int((taxi_row, taxi_col + 1) in self.wall_coords)
        obstacle_west = int((taxi_row, taxi_col - 1) in self.wall_coords)

        # Pad stations list to ensure we have 4 stations
        padded_stations = self.stations + [[0, 0]] * (4 - len(self.stations))

        # Check for passenger visibility in each direction
        passenger_loc = self.passenger_loc
        passenger_loc_north = int(passenger_loc == [taxi_row - 1, taxi_col])
        passenger_loc_south = int(passenger_loc == [taxi_row + 1, taxi_col])
        passenger_loc_east = int(passenger_loc == [taxi_row, taxi_col + 1])
        passenger_loc_west = int(passenger_loc == [taxi_row, taxi_col - 1])
        passenger_loc_middle = int(passenger_loc == [taxi_row, taxi_col])
        passenger_look = passenger_loc_north or passenger_loc_south or passenger_loc_east or passenger_loc_west or passenger_loc_middle

        # Check for destination visibility in each direction
        destination = self.destination
        destination_loc_north = int(destination == [taxi_row - 1, taxi_col])
        destination_loc_south = int(destination == [taxi_row + 1, taxi_col])
        destination_loc_east = int(destination == [taxi_row, taxi_col + 1])
        destination_loc_west = int(destination == [taxi_row, taxi_col - 1])
        destination_loc_middle = int(destination == [taxi_row, taxi_col])
        destination_look = destination_loc_north or destination_loc_south or destination_loc_east or destination_loc_west or destination_loc_middle

        # Construct the state tuple
        state = (
            taxi_row - 1,
            taxi_col - 1,
            padded_stations[0][0] - 1, padded_stations[0][1] - 1,
            padded_stations[1][0] - 1, padded_stations[1][1] - 1,
            padded_stations[2][0] - 1, padded_stations[2][1] - 1,
            padded_stations[3][0] - 1, padded_stations[3][1] - 1,
            obstacle_north, obstacle_south, obstacle_east, obstacle_west,
            passenger_look, destination_look
        )

        return state

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42) if torch.cuda.is_available() else None

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Neural Network for DQN
class DQNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=64):
        super(DQNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Experience Replay Buffer
class ReplayBuffer:
    def __init__(self, buffer_size):
        self.buffer = deque(maxlen=buffer_size)

    def add(self, obs, action, reward, next_obs, done):
        self.buffer.append((obs, action, reward, next_obs, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        observations, actions, rewards, next_observations, dones = zip(*batch)

        # Convert to numpy arrays first for better performance
        observations = np.array(observations, dtype=np.float32)
        actions = np.array(actions, dtype=np.int64)
        rewards = np.array(rewards, dtype=np.float32)
        next_observations = np.array(next_observations, dtype=np.float32)
        dones = np.array(dones, dtype=np.float32)

        return (
            torch.tensor(observations).to(device),
            torch.tensor(actions).to(device),
            torch.tensor(rewards).to(device),
            torch.tensor(next_observations).to(device),
            torch.tensor(dones).to(device)
        )

    def __len__(self):
        return len(self.buffer)

# DQN Agent updated for once-per-episode learning and epsilon decay per episode
class DQNAgent:
    def __init__(self, state_size, action_size, buffer_size=10000, batch_size=64,
                 gamma=0.99, learning_rate=0.001, tau=0.001):
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = batch_size
        self.gamma = gamma  # discount factor
        self.tau = tau      # for soft update of target network

        # Q-Networks (policy and target)
        self.policy_net = DQNetwork(state_size, action_size).to(device)
        self.target_net = DQNetwork(state_size, action_size).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()  # Target network is not trained directly

        # Optimizer
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)

        # Replay buffer
        self.memory = ReplayBuffer(buffer_size)

        # Exploration parameters
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9999  # Decay applied once per episode

    def act(self, obs, eval_mode=False):
        """Select an action using epsilon-greedy policy"""
        if (not eval_mode) and random.random() < self.epsilon:
            return random.randrange(self.action_size)

        obs = torch.FloatTensor(obs).unsqueeze(0).to(device)
        self.policy_net.eval()
        with torch.no_grad():
            action_values = self.policy_net(obs)
        self.policy_net.train()

        return torch.argmax(action_values, dim=1).item()

    def step(self, obs, action, reward, next_obs, done):
        """Add experience to memory.
           Epsilon decay is now handled once per episode in the training loop."""
        self.memory.add(obs, action, reward, next_obs, done)

    def learn(self):
        """Perform a single learning step using a batch of experience tuples"""
        # Sample from memory
        observations, actions, rewards, next_observations, dones = self.memory.sample(self.batch_size)

        # Get Q values for current states
        Q_expected = self.policy_net(observations).gather(1, actions.unsqueeze(1)).squeeze(1)

        # Get max Q values for next states from target model
        with torch.no_grad():
            Q_targets_next = self.target_net(next_observations).max(1)[0]

        # Compute target Q values
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        # Calculate loss
        loss = nn.MSELoss()(Q_expected, Q_targets)

        # Minimize loss
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1)
        self.optimizer.step()

        # Soft update target network
        self.soft_update()

    def soft_update(self):
        """Soft update of target network parameters"""
        for target_param, policy_param in zip(self.target_net.parameters(), self.policy_net.parameters()):
            target_param.data.copy_(self.tau * policy_param.data + (1.0 - self.tau) * target_param.data)

    def save(self, filename):
        """Save the model"""
        torch.save({
            'policy_model_state_dict': self.policy_net.state_dict(),
            'target_model_state_dict': self.target_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'epsilon': self.epsilon
        }, filename)

    def load(self, filename):
        """Load the model"""
        checkpoint = torch.load(filename)
        self.policy_net.load_state_dict(checkpoint['policy_model_state_dict'])
        self.target_net.load_state_dict(checkpoint['target_model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.epsilon = checkpoint['epsilon']

# Function to preprocess the state for the neural network
def preprocess_state(state):
    """Convert tuple state to numpy array for neural network input"""
    return np.array(state, dtype=np.float32)

# Training function updated to perform learning once per episode and decay epsilon once per episode
def train_dqn(env, agent, num_episodes=100000, max_steps=1000, print_every=100):
    """Train DQN agent with one update per episode and epsilon decay applied once per episode"""
    scores = []
    recent_scores = deque(maxlen=100)  # Last 100 scores for monitoring improvement

    for episode in tqdm(range(1, num_episodes+1)):
        # Reset environment at start of episode
        env = Maze(random.randint(5, 10))
        obs = preprocess_state(env.get_state())
        score = 0

        for t in range(max_steps):
            # Select an action
            action = agent.act(obs)

            # Take action and observe next state and reward
            next_obs, reward, done, _, _ = env.step(action)
            next_obs = preprocess_state(next_obs)

            # Store experience (learning is not done here)
            agent.step(obs, action, reward, next_obs, done)

            # Update state and score
            obs = next_obs
            score += reward

            if done:
                break

        # Perform a single parameter update at the end of the episode if possible
        if len(agent.memory) >= agent.batch_size:
            agent.learn()

        # Decay epsilon once per episode
        agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)

        scores.append(score)
        recent_scores.append(score)

        if episode % print_every == 0:
            print(f"Episode {episode}/{num_episodes} | Average Score: {np.mean(recent_scores):.2f} | Epsilon: {agent.epsilon:.4f}")
            # Save a checkpoint
            agent.save(f"dqn_checkpoint_ep{episode}.pt")

    # Save final model after training
    agent.save("dqn_final.pt")

    return scores

# Evaluation function
def evaluate(env, agent, num_episodes=10, render=False):
    """Evaluate a trained agent"""
    total_rewards = []

    for episode in range(num_episodes):
        # Reset environment
        env = Maze(random.randint(5, 10))
        obs = preprocess_state(env.get_state())
        total_reward = 0
        done = False
        step = 0

        while not done and step < 1000:
            # Select an action (no exploration)
            action = agent.act(obs, eval_mode=True)

            # Take action
            next_obs, reward, done, _, _ = env.step(action)
            next_obs = preprocess_state(next_obs)

            # Update state and reward
            obs = next_obs
            total_reward += reward
            step += 1

            # Render if requested
            if render:
                clear_screen()  # Assuming you have a function to clear the screen
                display_state(env)  # And a function to display the maze
                print(f"Action: {action}")
                print(f"Reward: {reward:.1f}")
                print(f"Total Reward: {total_reward:.1f}")
                input("Press Enter to continue...")

        total_rewards.append(total_reward)
        print(f"Episode {episode+1}: Total Reward = {total_reward:.2f}, Steps = {step}")

    print(f"Average Reward over {num_episodes} episodes: {np.mean(total_rewards):.2f}")
    return total_rewards

# Main function to train and evaluate DQN agent with once-per-episode updates
def main_func():
    # Create environment and agent
    env = Maze(6)  # Initialize with maze size 6 (ensure Maze is defined)
    state_size = len(env.get_state())
    action_size = 6  # Example: South, North, East, West, Pickup, Dropoff

    agent = DQNAgent(state_size, action_size)

    print("Training agent...")
    scores = train_dqn(env, agent)

    # Plot training progress
    plt.figure(figsize=(10, 6))
    plt.plot(scores)
    plt.title('DQN Training Progress')
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.savefig('dqn_training_progress.png')
    plt.show()

    # Plot moving average of scores for smoother visualization
    window_size = 100
    moving_avg = [np.mean(scores[max(0, i-window_size):(i+1)]) for i in range(len(scores))]

    plt.figure(figsize=(10, 6))
    plt.plot(moving_avg)
    plt.title('DQN Training Progress (Moving Average)')
    plt.xlabel('Episode')
    plt.ylabel('Average Score (over 100 episodes)')
    plt.savefig('dqn_training_progress_avg.png')
    plt.show()

    print("\nEvaluating agent...")
    eval_rewards = evaluate(env, agent, num_episodes=10, render=False)

    print("\nTraining complete!")

main_func()


Using device: cuda
Training agent...


  0%|          | 0/100000 [00:00<?, ?it/s]

Episode 100/100000 | Average Score: -4866.95 | Epsilon: 0.9900
Episode 200/100000 | Average Score: -4835.35 | Epsilon: 0.9802
Episode 300/100000 | Average Score: -4768.80 | Epsilon: 0.9704
Episode 400/100000 | Average Score: -4715.50 | Epsilon: 0.9608
Episode 500/100000 | Average Score: -4751.45 | Epsilon: 0.9512
Episode 600/100000 | Average Score: -4638.55 | Epsilon: 0.9418
Episode 700/100000 | Average Score: -4564.85 | Epsilon: 0.9324
Episode 800/100000 | Average Score: -4537.40 | Epsilon: 0.9231
Episode 900/100000 | Average Score: -4464.30 | Epsilon: 0.9139
Episode 1000/100000 | Average Score: -4385.65 | Epsilon: 0.9048
Episode 1100/100000 | Average Score: -4334.75 | Epsilon: 0.8958
Episode 1200/100000 | Average Score: -4314.10 | Epsilon: 0.8869
Episode 1300/100000 | Average Score: -4244.45 | Epsilon: 0.8781
Episode 1400/100000 | Average Score: -4157.20 | Epsilon: 0.8694
Episode 1500/100000 | Average Score: -4160.30 | Epsilon: 0.8607
Episode 1600/100000 | Average Score: -4120.85 | E